# 1. BasePromptTemplate
This is the base class from which all prompt templates inherit. It provides common methods for handling prompts.

In [6]:
from langchain.prompts import BasePromptTemplate
from pydantic import Field

class MyPromptTemplate(BasePromptTemplate):
    template: str = Field(...)
    input_variables: list = Field(...)

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

    def format_prompt(self, **kwargs) -> str:
        return self.format(**kwargs)

template = "Summarize the following text: {text}"

# Create an instance of MyPromptTemplate with the template and input variables
prompt = MyPromptTemplate(template=template, input_variables=["text"])

# Now you can use the format method without any issues
formatted_prompt = prompt.format(text="The quick brown fox jumps over the lazy dog.")
print(formatted_prompt)  # Output: Summarize the following text: The quick brown fox jumps over the lazy dog.


Summarize the following text: The quick brown fox jumps over the lazy dog.


# 2. AIMessagePromptTemplate
This is specifically for AI-generated messages in chat-based applications. It's useful when interacting with AI models.

In [3]:
from langchain.prompts.chat import AIMessagePromptTemplate

ai_template = AIMessagePromptTemplate.from_template("AI: {response}")
ai_message = ai_template.format(response="Here is your answer.")
print(ai_message.content)  # Output: AI: Here is your answer.


AI: Here is your answer.


# 3. BaseChatPromptTemplate
This is a base class for chat-based prompt templates. It's an abstract class that defines the structure for chat prompt templates.

In [7]:
from langchain.prompts.chat import BaseChatPromptTemplate
from pydantic import Field

class MyChatPromptTemplate(BaseChatPromptTemplate):
    input_variables: list = Field(default_factory=list)

    def format_messages(self, **kwargs):
        return [f"Chatbot says: {kwargs['user_message']}"]

# Create an instance of MyChatPromptTemplate
template = MyChatPromptTemplate(input_variables=["user_message"])

# Test the format_messages method
print(template.format_messages(user_message="Hello"))  # Output: ['Chatbot says: Hello']


['Chatbot says: Hello']


# 4. BaseMessagePromptTemplate
This class handles the structure of individual message prompts and is a base class for specific message templates.

In [ ]:
from langchain.prompts.chat import BaseMessagePromptTemplate
from pydantic import Field

class CustomMessagePromptTemplate(BaseMessagePromptTemplate):
    input_variables: list = Field(default_factory=list)

    def format(self, **kwargs):
        return f"Message: {kwargs['text']}"

    def format_messages(self, **kwargs):
        return [self.format(**kwargs)]


# Create an instance of CustomMessagePromptTemplate
template = CustomMessagePromptTemplate(input_variables=["text"])

# Test the format method
print(template.format(text="Welcome!"))  # Output: Message: Welcome!

# Test the format_messages method
print(template.format_messages(text="Welcome!"))  # Output: ['Message: Welcome!']


Message: Welcome!
['Message: Welcome!']


# 5. BaseStringMessagePromptTemplate
This is a more specific class dealing with string-based message prompts. It is typically inherited when handling string messages.

In [11]:
from langchain.prompts.chat import ChatMessagePromptTemplate

# Specify the required role (e.g., "user")
user_template = ChatMessagePromptTemplate.from_template(
    template="User: {message}",
    role="user"  # Define the role here
)

# Format the message with the given input
formatted_message = user_template.format(message="What is the weather today?")

# Print the content of the formatted message
print(formatted_message.content)  # Expected Output: User: What is the weather today?


User: What is the weather today?


# 6. ChatMessagePromptTemplate
This template is for a single chat message prompt, useful when working with multi-turn conversations.

In [13]:
from langchain.prompts.chat import ChatMessagePromptTemplate

# Define a role, in this case, "user"
user_template = ChatMessagePromptTemplate.from_template("User: {message}", role="user")

# Format the message
formatted_message = user_template.format(message="What is the weather today?")

# Output the formatted message content
print(formatted_message.content)  # Output: User: What is the weather today?


User: What is the weather today?


# 7. ChatPromptTemplate
This is used to create a conversation-style prompt with multiple messages from different speakers (e.g., human, AI).

In [8]:
from langchain.prompts.chat import ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [HumanMessagePromptTemplate.from_template("User: {user_message}"),
     AIMessagePromptTemplate.from_template("AI: {ai_response}")]
)

prompt = chat_prompt.format_messages(user_message="How are you?", ai_response="I'm good, thanks!")
print([msg.content for msg in prompt])  
# Output: ['User: How are you?', "AI: I'm good, thanks!"]


['User: How are you?', "AI: I'm good, thanks!"]


# 8. HumanMessagePromptTemplate
This template formats messages from the human user. It is used in chat-based prompts.

In [9]:
from langchain.prompts.chat import HumanMessagePromptTemplate

user_template = HumanMessagePromptTemplate.from_template("Human: {text}")
print(user_template.format(text="Hello!").content)  # Output: Human: Hello!


Human: Hello!


# 9. MessagesPlaceholder
This placeholder allows you to insert dynamic content into a message.

In [15]:
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder

# Define a placeholder for previous messages
placeholder = MessagesPlaceholder(variable_name="previous_messages")

# Create a prompt template that includes the placeholder
prompt = ChatPromptTemplate.from_messages([placeholder])

# Format the template by passing previous messages as a variable
formatted_prompt = prompt.format(previous_messages=[
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "How can I help you?"}
])

# Since formatted_prompt may be a string, simply print it directly.
print(formatted_prompt)


Human: Hello
AI: How can I help you?


# 10. SystemMessagePromptTemplate
This template defines system messages, which set instructions or context for the conversation.

In [11]:
from langchain.prompts.chat import SystemMessagePromptTemplate

system_template = SystemMessagePromptTemplate.from_template("System: You are a helpful assistant.")
print(system_template.format().content)  # Output: System: You are a helpful assistant.


System: You are a helpful assistant.


# 11. FewShotChatMessagePromptTemplate
This is a specialized prompt template for few-shot learning, where a model is provided with a few examples to understand a task.

In [16]:
from langchain.prompts import ChatPromptTemplate

# Create a few-shot prompt manually
examples = [
    {"input": "How are you?", "output": "I'm fine, thanks!"},
    {"input": "What is your name?", "output": "I am a chatbot."}
]

input_template = "User: {input}\n"
output_template = "AI: {output}\n"

# Combine examples into a single prompt
few_shot_prompt = "".join(
    input_template.format(input=example["input"]) + output_template.format(output=example["output"])
    for example in examples
)

# Add the new input to the prompt
new_input = "What's your name?"
formatted_prompt = few_shot_prompt + input_template.format(input=new_input)

# Output the formatted prompt
print(formatted_prompt)


User: How are you?
AI: I'm fine, thanks!
User: What is your name?
AI: I am a chatbot.
User: What's your name?



# 12. FewShotPromptTemplate
Similar to the few-shot chat version, but designed for general non-chat applications where you give examples.

In [21]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Define the example prompt template for translation
example_prompt_template = PromptTemplate(
    template="Example: {task} for '{data}' is '{output}'.",
    input_variables=["task", "data", "output"]
)

# Create a FewShotPromptTemplate for language translation
few_shot_template = FewShotPromptTemplate(
    examples=[
        {"task": "translate", "data": "Hello, how are you?", "output": "Hola, ¿cómo estás?"},
        {"task": "translate", "data": "Thank you very much!", "output": "¡Muchas gracias!"},
    ],
    example_prompt=example_prompt_template,  # Use example_prompt instead of example_prompt_template
    input_variables=["task", "data"],
    prefix="You are a language translator.",
    suffix="Complete the task: {task} for '{data}'."
)

# Format the prompt with new input data
prompt = few_shot_template.format(task="translate", data="What is your name?")
print(prompt)


You are a language translator.

Example: translate for 'Hello, how are you?' is 'Hola, ¿cómo estás?'.

Example: translate for 'Thank you very much!' is '¡Muchas gracias!'.

Complete the task: translate for 'What is your name?'.


# 14. PipelinePromptTemplate
This template allows combining multiple prompts into a pipeline.

* Example 1: Greeting, Product Information, and Closing

In [35]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

# Define the component prompt templates
greeting_prompt = PromptTemplate.from_template("Good morning How can I assist you today?")
product_info_prompt = PromptTemplate.from_template("You're interested in learning more about our products. Here are the details about our latest product {product}.")
closing_prompt = PromptTemplate.from_template("If you have any other questions, feel free to ask. Have a great day!")

# Define the final prompt template that combines all parts
full_template = """{greeting}

{content}

{closing}"""
full_prompt = PromptTemplate.from_template(full_template)

# Assemble the pipeline prompts
pipeline_prompts = [
    ("greeting", greeting_prompt),
    ("content", product_info_prompt),
    ("closing", closing_prompt)
]

# Create the PipelinePromptTemplate
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=pipeline_prompts)

# Example usage for product information
formatted_prompt = pipeline_prompt.format(product="Educative Bot")
print(formatted_prompt)

Good morning How can I assist you today?

You're interested in learning more about our products. Here are the details about our latest product Educative Bot.

If you have any other questions, feel free to ask. Have a great day!


* Example 2: Troubleshooting Prompt

In [36]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

# Define the component prompt templates
greeting_prompt = PromptTemplate.from_template("Good morning How can I assist you today?")
troubleshooting_prompt = PromptTemplate.from_template("It seems you're having trouble with {product}. Let me guide you through some steps to resolve this.")
closing_prompt = PromptTemplate.from_template("If you have any other questions, feel free to ask. Have a great day!")

# Define the final prompt template that combines all parts
troubleshoot_template = """{greeting}

{trouble}

{closing}"""
full_prompt = PromptTemplate.from_template(troubleshoot_template)

# Assemble the pipeline prompts
pipeline_prompts = [
    ("greeting", greeting_prompt),
    ("trouble", troubleshooting_prompt),
    ("closing", closing_prompt)
]

# Create the PipelinePromptTemplate
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=pipeline_prompts)

# Example usage for troubleshooting
formatted_prompt = pipeline_prompt.format(product="Educative Bot")
print(formatted_prompt)

Good morning How can I assist you today?

It seems you're having trouble with Educative Bot. Let me guide you through some steps to resolve this.

If you have any other questions, feel free to ask. Have a great day!


* Example 3: Impersonation and Interaction

In [ ]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

# Define the component prompt templates
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

# Define the final prompt template that combines all parts
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

# Assemble the pipeline prompts
pipeline_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]

# Create the PipelinePromptTemplate
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=pipeline_prompts)

# Example usage
formatted_prompt = pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Tesla",
    input="What's your favorite social media site?"
)
print(formatted_prompt)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


# 15. PromptTemplate
The core template for non-chat-based prompting. It handles dynamic string generation.

In [16]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("Classify the following text: {text}")
prompt = template.format(text="apple")
print(prompt)  # Output: Classify the following text: apple


Classify the following text: apple


* Question-Answer Prompt Template

In [38]:
prompt_template = """
Instruction: Answer the question based on the context below. If you cannot answer the question with the given context, answer with "I don't know."

Context: {context}

Query: {query}

Answer:
"""

# Example usage
context = "Codecademy is an interactive online learning platform offering courses in various programming languages and tech skills."
query = "What types of courses does Codecademy offer?"
prompt = prompt_template.format(context=context, query=query)
print(prompt)


Instruction: Answer the question based on the context below. If you cannot answer the question with the given context, answer with "I don't know."

Context: Codecademy is an interactive online learning platform offering courses in various programming languages and tech skills.

Query: What types of courses does Codecademy offer?

Answer:



* Text Summarization Prompt Template

In [49]:
from langchain.prompts import PromptTemplate

# Summarize the text
text_to_summarize = "The quick brown fox jumps over the lazy dog."
summary = "The sentence describes a quick brown fox jumping over a lazy dog. It is often used as an example in typing and typing games due to its use of all the alphabet letters."
print("Summary:", summary)

# Define the prompt template
prompt_template = PromptTemplate.from_template("Task: {task}, Data: {data}, Output: {output}")

# Format the prompt
prompt = prompt_template.format(task="classify", data="apple", output="fruit")
print(prompt)  # Output: Task: classify, Data: apple, Output: fruit

# Example classification function
def classify_text(text):
    return "fruit" if text.lower() == "apple" else "unknown"

# Classifying the text
classification_text = "apple"
classification_result = classify_text(classification_text)
print(f"Classification result for '{classification_text}': {classification_result}")

# Define the feedback request template correctly
feedback_template = PromptTemplate.from_template("We would love to hear your thoughts on {service}. Please provide your feedback:")
formatted_feedback_request = feedback_template.format(service="our new mobile app")
print(formatted_feedback_request)  # Output: "We would love to hear your thoughts on our new mobile app. Please provide your feedback:"

Summary: The sentence describes a quick brown fox jumping over a lazy dog. It is often used as an example in typing and typing games due to its use of all the alphabet letters.
Task: classify, Data: apple, Output: fruit
Classification result for 'apple': fruit
We would love to hear your thoughts on our new mobile app. Please provide your feedback:


# 16. StringPromptTemplate
This is a string-based version of prompt templates.

In [62]:
from langchain_core.prompts.string import StringPromptTemplate
from langchain_core.runnables import RunnableLambda

# Define a simple prompt template
class CustomStringPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        return f"{kwargs['name']}, welcome!"  # Custom formatting logic

# Create an instance of the custom template
template = CustomStringPromptTemplate(
    input_variables=["name"],
    input_types={"name": str},
)

# Create a function to use with the template
async def generate_greeting(name: str) -> str:
    formatted_prompt = template.format(name=name)
    return formatted_prompt

# Create a runnable from the function
greeting_runnable = RunnableLambda(generate_greeting)

# Example invocation using `ainvoke` for asynchronous execution
output = await greeting_runnable.ainvoke({"name": "Alice"})  # Use ainvoke here
print(output)  # Outputs: "Alice, welcome!"


{'name': 'Alice'}, welcome!


In [ ]:
z